In [2]:
# Do all imports and installs here
import pandas as pd
import os
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from datetime import datetime, timedelta
import json
from pyspark.sql.functions import desc, monotonically_increasing_id, udf, to_date, from_unixtime, trim, col
from custom_udf import *
from etl import *
spark = SparkSession.builder.config(
    "spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0"
).getOrCreate()
input_data = "./data"
output_data = "./data/processed_data/"

:: loading settings :: url = jar:file:/Users/yugesh/opt/anaconda3/envs/airflow/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/yugesh/.ivy2/cache
The jars for the packages stored in: /Users/yugesh/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8dc25521-1a54-4ce1-8617-a25527f50303;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;2.7.0 in central
	found org.apache.hadoop#hadoop-common;2.7.0 in central
	found org.apache.hadoop#hadoop-annotations;2.7.0 in central
	found com.google.guava#guava;11.0.2 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found commons-cli#commons-cli;1.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found xmlenc#xmlenc;0.52 in central
	found commons-httpclient#commons-httpclient;3.1 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.4 in central
	found commons-io#commons-io;2.4 in central
	found commons-net#commons-net;3.1 in central
	found commons-collections#commons-colle

## Transform airports etl

In [2]:
airport_df =  process_airports_data(
        spark=spark, input_data=input_data, output_data=output_data
    )

In [3]:
airport_df.printSchema()

root
 |-- airport_id: long (nullable = false)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- type: string (nullable = true)



In [4]:
airport_df.show(10)

+----------+--------------------+------------+-----------+-------------+
|airport_id|                name|        city|      state|         type|
+----------+--------------------+------------+-----------+-------------+
|         0|Perrotti Skyranch...|     Berwick|      MAINE|small_airport|
|         1|Pitcock Rosillos ...|    Marathon|      TEXAS|small_airport|
|         3|Lewis Air Service...|      Leland|MISSISSIPPI|small_airport|
|         5|Copper Basin Airport|      Mackay|      IDAHO|small_airport|
|         6|   Applegate Airport|  Queen City|   MISSOURI|small_airport|
|         7|    Propwash Airport|      Justin|      TEXAS|small_airport|
|        10|Double Tree Farm ...|  Marysville| CALIFORNIA|small_airport|
|        11|      Morrison Field|Bristolville|       OHIO|small_airport|
|        12|   Taylor's Air Park|      Joshua|      TEXAS|small_airport|
|        16|Wiley's Seaplane ...| Lake Oswego|     OREGON|seaplane_base|
+----------+--------------------+------------+-----

In [5]:
airport_df.select("state").show()

+------------+
|       state|
+------------+
|       MAINE|
|       TEXAS|
| MISSISSIPPI|
|       IDAHO|
|    MISSOURI|
|       TEXAS|
|  CALIFORNIA|
|        OHIO|
|       TEXAS|
|      OREGON|
|       TEXAS|
|PENNSYLVANIA|
|      OREGON|
|       TEXAS|
|      ALASKA|
|       TEXAS|
|  CALIFORNIA|
|     FLORIDA|
| N. CAROLINA|
|       TEXAS|
+------------+
only showing top 20 rows



Traceback (most recent call last):
  File "/Users/yugesh/opt/anaconda3/envs/airflow/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/Users/yugesh/opt/anaconda3/envs/airflow/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/yugesh/opt/anaconda3/envs/airflow/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 663, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/yugesh/opt/anaconda3/envs/airflow/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [6]:
airport_df.select("type").distinct().show()

+--------------+
|          type|
+--------------+
| seaplane_base|
|medium_airport|
| small_airport|
| large_airport|
+--------------+



In [7]:
airport_df.count()

15148

In [8]:
test = airport_df.dropDuplicates()
test.count()

15148

In [ ]:
airport_df = (
    airport_df.withColumn(
        "name",
        remove_whitespace_udf(airport_df.name)
        if airport_df.name.isNotNull
        else airport_df.name,
    )
    .withColumn(
        "city",
        remove_whitespace_udf(airport_df.city)
        if airport_df.city.isNotNull
        else airport_df.city,
    )
    .withColumn(
        "state",
        remove_whitespace_udf(airport_df.state)
        if airport_df.state.isNotNull
        else airport_df.state,
    )
    .withColumn(
        "type",
        remove_whitespace_udf(airport_df.type)
        if airport_df.type.isNotNull
        else airport_df.type,
    )
)

In [ ]:
used_airports = ["seaplane_base","medium_airport", "small_airport", "large_airport" ]
test = airport_df.filter(airport_df.type.isin(used_airports))

In [ ]:
test.select("type").distinct().show()

In [ ]:
test = test.filter( col("type") != "balloonport" or col("type") !=)

In [ ]:
test.show()

In [ ]:
df.select("type").distinct().show()

## Immigration data etl

In [ ]:

df = process_immigration_data(
    spark=spark, input_data=input_data, output_data=output_data
)

In [ ]:
df.printSchema()

In [ ]:
df.show(10)

In [ ]:
# df.write.mode("overwrite").parquet("./data/processed_data")

## demographics data etl

In [ ]:
df =  process_cities_demographics(
        spark=spark, input_data=input_data, output_data=output_data
    )

In [6]:
df.printSchema()

root
 |-- airport_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- type: string (nullable = true)



In [17]:
df = spark.read.parquet("./data/processed_data/airport")
df.count()

15148

In [18]:
df = spark.read.parquet("./data/processed_data/demographic")
df.count()

596

In [26]:
spark.read.csv("s3a://smart-advertisement/data/csv_data/us_cities_demographics.csv")

21/12/08 11:40:30 WARN FileSystem: Failed to initialize fileystem s3a://smart-advertisement/data/csv_data/us_cities_demographics.csv: java.lang.NumberFormatException: For input string: "64M"
21/12/08 11:40:30 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://smart-advertisement/data/csv_data/us_cities_demographics.csv.
java.lang.NumberFormatException: For input string: "64M"
	at java.lang.NumberFormatException.forInputString(NumberFormatException.java:65)
	at java.lang.Long.parseLong(Long.java:589)
	at java.lang.Long.parseLong(Long.java:631)
	at org.apache.hadoop.conf.Configuration.getLong(Configuration.java:1565)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:248)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3469)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.had

Py4JJavaError: An error occurred while calling o94.csv.
: java.lang.NumberFormatException: For input string: "64M"
	at java.lang.NumberFormatException.forInputString(NumberFormatException.java:65)
	at java.lang.Long.parseLong(Long.java:589)
	at java.lang.Long.parseLong(Long.java:631)
	at org.apache.hadoop.conf.Configuration.getLong(Configuration.java:1565)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:248)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3469)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:747)
	at scala.collection.immutable.List.

In [24]:
df = spark.read.parquet("s3a://smart-advertisement/processed_data/airport/part-00000-363ac92b-32c9-4ba2-b27b-145a42031373-c000.snappy.parquet")
df.count()

21/12/08 11:38:56 WARN FileSystem: Failed to initialize fileystem s3a://smart-advertisement/processed_data/airport/part-00000-363ac92b-32c9-4ba2-b27b-145a42031373-c000.snappy.parquet: java.lang.NumberFormatException: For input string: "64M"
21/12/08 11:38:56 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://smart-advertisement/processed_data/airport/part-00000-363ac92b-32c9-4ba2-b27b-145a42031373-c000.snappy.parquet.
java.lang.NumberFormatException: For input string: "64M"
	at java.lang.NumberFormatException.forInputString(NumberFormatException.java:65)
	at java.lang.Long.parseLong(Long.java:589)
	at java.lang.Long.parseLong(Long.java:631)
	at org.apache.hadoop.conf.Configuration.getLong(Configuration.java:1565)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:248)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3469)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java

Py4JJavaError: An error occurred while calling o84.parquet.
: java.lang.NumberFormatException: For input string: "64M"
	at java.lang.NumberFormatException.forInputString(NumberFormatException.java:65)
	at java.lang.Long.parseLong(Long.java:589)
	at java.lang.Long.parseLong(Long.java:631)
	at org.apache.hadoop.conf.Configuration.getLong(Configuration.java:1565)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:248)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3469)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:747)
	at scala.collection.immutable.L

In [20]:
df.printSchema()

root
 |-- cicid: long (nullable = true)
 |-- origin_city: string (nullable = true)
 |-- traveled_from: string (nullable = true)
 |-- arrived_city: string (nullable = true)
 |-- us_address: string (nullable = true)
 |-- arrival_date: date (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- transportation_mode: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- visa_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- airline: string (nullable = true)



In [12]:
df.show()

+----------+--------------------+---------------+------------+-------------+
|airport_id|                 abc|           city|       state|         type|
+----------+--------------------+---------------+------------+-------------+
|         0|Perrotti Skyranch...|        Berwick|       MAINE|small_airport|
|         1|Pitcock Rosillos ...|       Marathon|       TEXAS|small_airport|
|         3|Lewis Air Service...|         Leland| MISSISSIPPI|small_airport|
|         5|Copper Basin Airport|         Mackay|       IDAHO|small_airport|
|         6|   Applegate Airport|     Queen City|    MISSOURI|small_airport|
|         7|    Propwash Airport|         Justin|       TEXAS|small_airport|
|        10|Double Tree Farm ...|     Marysville|  CALIFORNIA|small_airport|
|        11|      Morrison Field|   Bristolville|        OHIO|small_airport|
|        12|   Taylor's Air Park|         Joshua|       TEXAS|small_airport|
|        16|Wiley's Seaplane ...|    Lake Oswego|      OREGON|seaplane_base|